In [1]:
import numpy as np
import random

In [2]:
class Environement:
    def __init__(self):
        self.board = np.full((3, 3), -1)
        self.done = False
        self.player1turn = True 
    
    def step(self,move : int,):
        player = int(self.player1turn)
        one_hot_move = np.zeros(9)
        one_hot_move[move]=1
        coords = (move // 3, move % 3)
        if self.board[coords] != -1:
            raise Exception("Already occupied")
        self.board[coords] = player
        
    def reset(self):
        self.board = np.full((3, 3), -1)
        
    def check_win(self):
        player_val = int(self.player1turn)
        for i in range(3):
            if np.all(self.board[i, :] == player_val):  
                env.done = True
                return True
            if np.all(self.board[:, i] == player_val): 
                env.done = True
                return True
        if np.all(np.diag(self.board) == player_val): 
            env.done = True
            return True
        if np.all(np.diag(np.fliplr(self.board)) == player_val):
            env.done = True
            return True
        env.done = False 
        return False
        
    def print_board(self):
        for row in self.board:
            print("|".join(self.symbol(cell) for cell in row))
            print("-" * 5)
            
    def symbol(self,val):
        if val == 1:
            return "X"
        elif val == 0:
            return "O"
        else:
            return " "

In [3]:
env = Environement()
while not env.done:
    
    available_coords = list(zip(*np.where(env.board == -1)))
    available = [r * 3 + c for r, c in available_coords]
    if not available:
        print("It's a draw!")
        break

    move = random.choice(available)
    env.step(move)
    env.print_board()
    
    env.check_win()
    if env.done:
        print(f"Player {env.symbol(int(env.player1turn))} has won")
    env.player1turn = not env.player1turn
    input()
    
    
    

 | | 
-----
 | | 
-----
X| | 
-----


 | | 
-----
 |O| 
-----
X| | 
-----


 | | 
-----
X|O| 
-----
X| | 
-----


 | |O
-----
X|O| 
-----
X| | 
-----


X| |O
-----
X|O| 
-----
X| | 
-----
Player X has won
